In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from itertools import islice


In [2]:
# Reading in file and adding it to array to one hot encode. 
#file = open('samplemusic.txt')
file = open('train.txt')
tag = False
part_tag = False
array = []
tag_string = ''
while 1:
    char = file.read(1)  # read by character
    if not char: break
        
    # if char is beginning of tag
    if char == '<':
        part_tag = True
        tag_string += char
        
    # if char is end of tag
    elif char == '>':
        # Checks if > is part of <start>/<end> tags or lone char
        if tag_string == '<start' or tag_string == '<end':
            tag = False
            tag_string += char
            array.append(tag_string)
        # > is not part of tag, just lone char so we append it and reset the tag_string
        else: 
            array.append(char)
        tag_string = ''
        
    
    # checks if char is still part of tag
    elif part_tag == True:
        # If next char after < is either s or e, we assume this is a tag.
        if char == 's' or char == 'e':
            tag = True
            tag_string += char
        # If not, < is just a lone char so append both separately
        else:
            array.append(tag_string)
            array.append(char)
            tag_string = ''
        part_tag = False
        
    # If inside tag, keep adding chars to create full tag
    elif tag == True:
        tag_string += char
            
    # if char is not part of tag, just add char
    elif tag == False:
        array.append(char)
    
file.close()

In [3]:
# One-hot encoding of data
array = np.array(array).reshape(-1,1)
print('Shape or original data array:', array.shape)
print('Number of unique chars in array:', len(np.unique(array)))
onehot_encoder = OneHotEncoder(sparse=False)
X_data = onehot_encoder.fit_transform(array) # Contains One hot encoding of all chars in text 

# Example of inversing one hot to get letter
inverted = onehot_encoder.inverse_transform(X_data[0, :].reshape(1,-1))
print(inverted)
print('Shape of one-hot encoded data, X_data:', X_data.shape)
num_chars = X_data.shape[0]
num_hot_encoding = X_data.shape[1]

Shape or original data array: (185410, 1)
Number of unique chars in array: 153
[['<start>']]
Shape of one-hot encoded data, X_data: (185410, 153)


In [4]:
# Turning songs from data into chunks 
train_data = []
song = []
temp = []
for idx in range(len(X_data)):
    temp.append(X_data[idx]) # Add 100 chars into temp to create chunks 
    chara = onehot_encoder.inverse_transform(X_data[idx, :].reshape(1,-1))[0][0]
    # Checks if char is at the end of a song 
    #print(chara)
    if chara == '<end>':
        song.append(np.array(temp))
        train_data.append(np.array(song))
        temp = []
        song = []
    
    # If 100 chars in temp, add chunk to train_data
    if idx%100 == 99:
        song.append(np.array(temp))
        temp = []

train_data = np.array(train_data)
print(train_data.shape[0])

333


In [5]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length

        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True)

    def forward(self, x, *args):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True
        hidden = args[0][0]
        cell = args[0][1]
        # Reshape input
        x.view(1, self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)

        out, (hidden, cell) = self.lstm(x, (hidden,cell))
        return out.view(-1, num_classes), (hidden,cell)
    
    def init_hidden(self):
        if torch.cuda.is_available():
            hidden = torch.zeros(self.num_layers, 1, self.hidden_size).cuda()
            cell = torch.zeros(self.num_layers, 1, self.hidden_size).cuda()
        else:
            hidden = torch.zeros(self.num_layers, 1, self.hidden_size)
            cell = torch.zeros(self.num_layers, 1, self.hidden_size)


        return (Variable(hidden), Variable(cell))

In [6]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

CUDA is supported


In [31]:
torch.manual_seed(777)  # reproducibility
print(np.array(train_data[0][0]).shape)
num_classes = num_hot_encoding
input_size = num_hot_encoding  # one-hot size
hidden_size = num_hot_encoding  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 100  # One hundred sized chunks of text 
num_layers = 1  # one-layer rnn

# Instantiate RNN model
lstm = LSTM(num_classes, input_size, hidden_size, num_layers).to(device)
print(lstm)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.1)

total_loss = []
avg_chunk_loss = []

# Train the model
for epoch in range(2):
    N_chunk_loss = 0.0
    correct = 0
    total = 0
    song_count = 0
    for song in train_data:
        optimizer.zero_grad()
        hidden_cont = None
        for chunk_count, chunk in enumerate(song):
            targets = np.roll(chunk, -1) # Targets is the chunk left shifted once
            # If chunk is first one in song, init hidden state, else keep hidden state for next chunk
            if chunk_count == 0:
                (hidden, cell) = lstm.init_hidden() # Inits hidden state
            else:
                print('cont')
                (hidden, cell) = hidden_cont
            loss = 0
            
            letter = Variable(torch.Tensor(chunk)).view(1,-1,153).to(device)
            target_letter = Variable(torch.LongTensor(targets)).view(1,-1,153).to(device)
            output, (hidden, cell) = lstm(letter, (hidden, cell))
            hidden_cont = (hidden, cell)
            print('out:', torch.argmax(output, dim=1).size())
            print('targ:', torch.argmax(target_letter, dim=2).squeeze().size())
            loss += criterion(output, torch.argmax(target_letter, dim=2).squeeze())
            #for c in range(len(chunk)):
            #    letter = Variable(torch.Tensor(chunk[c])).view(1,1,153).to(device)
            #    target_letter = Variable(torch.LongTensor(targets[c])).view(1,1,153).to(device)
            #    output, (hidden, cell) = lstm(letter, (hidden,cell))
            #    hidden_cont = (hidden, cell)
            #    #print('output:', torch.argmax(output))
            #    #print('target:', torch.argmax(target_letter).view(1))
            #    loss += criterion(output, torch.argmax(target_letter).view(1))

            loss.backward()
            optimizer.step()
            N_chunk_loss += loss
            
            #Print the loss averaged over the last N mini-batches
            N_chunk_loss /= 1
            #print('Epoch %d, average minibatch loss: %.3f' % (epoch + 1, N_chunk_loss))
            # Add the averaged loss over N minibatches and reset the counter
            avg_chunk_loss.append(N_chunk_loss)
            N_chunk_loss = 0.0
        print("Finished", song_count + 1, "/333 songs of training")
        song_count += 1
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))

print("Learning finished!")

(100, 153)
LSTM(
  (lstm): LSTM(153, 153, batch_first=True)
)
out: torch.Size([100])
targ: torch.Size([100])
cont
out: torch.Size([100])
targ: torch.Size([100])


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.